https://www.youtube.com/watch?v=07EiCpdhtDE&t=615s


In [8]:
from dask.distributed import Client

client = Client()
client


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 80,Total memory: 754.54 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33781,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 80
Started: Just now,Total memory: 754.54 GiB
Comm: tcp://127.0.0.1:33455,Total threads: 8
Dashboard: http://127.0.0.1:46091/status,Memory: 75.45 GiB
Nanny: tcp://127.0.0.1:34429,


In [2]:
import dask.array as da

x = da.random.random((10000, 10000), chunks="16 MiB")
x


dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(1448, 1448), chunktype=numpy.ndarray>

In [3]:
y = (x + x.T) - x.mean(axis=0)
y.sum().compute()


50000645.6327875

In [6]:
import time
import numpy as np


def load(x):
    time.sleep(0.2)
    return np.arange(1_000_000) + x


def process(x):
    time.sleep(0.1)
    return x + 1


def save(x):
    time.sleep(0.4)
    return None


In [7]:
%%time

inputs = range(10)

for i in inputs:
    x = load(i)
    y = process(x)
    save(y)

CPU times: user 204 ms, sys: 29.2 ms, total: 234 ms
Wall time: 7.07 s


In [9]:
%%time

inputs = range(10)
futures = []

for i in inputs:
    x = client.submit(load, i)
    y = client.submit(process, x)
    z = client.submit(save, y)
    futures.append(z)

results = [future.result() for future in futures]

CPU times: user 137 ms, sys: 20.3 ms, total: 157 ms
Wall time: 792 ms


In [10]:
L = [client.submit(load, i) for i in range(200)]
L2 = client.map(process, L)
L3 = client.map(save, L2)


In [11]:
L[:5]


[<Future: finished, type: numpy.ndarray, key: load-50c63bbfc41a89251a3e622f15b4cb93>,
 <Future: finished, type: numpy.ndarray, key: load-2c25fd75f9f270ff73bcdb014cb4406d>,
 <Future: finished, type: numpy.ndarray, key: load-a6da951aba520f365f71cc9d64f35d20>,
 <Future: finished, type: numpy.ndarray, key: load-4a0811fbfe25e0a0ae6eb3bec47a5603>,
 <Future: finished, type: numpy.ndarray, key: load-a85a279b4f6fb97bc1c05e985806d400>]

In [12]:
del L


In [13]:
del L2


In [14]:
del L3


In [15]:
client.shutdown()
